In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn import preprocessing
import seaborn as sns
from sklearn.metrics import roc_auc_score, matthews_corrcoef, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE

import time
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [13]:
train_df = pd.read_csv("input.csv") 
#tanish_df = pd.read_csv("Data/input_tanish.csv")

#mappings = {1: 0, 2: 1, 3:2}

#train_df.label = [mappings[item] for item in train_df.label]
train_df

,Unnamed: 0,patient_id,sequence,label
0,0,Z8170308,medications allergies she is allergic to oxyco...,1
1,1,Z9021665,follow commands follows step commands follow c...,0
2,2,Z8485823,cancer brother testicular cancer u coronary ar...,1
3,3,Z10380050,are progressing faster he is very motivated to...,2
4,4,Z6985118,relation age of onset u cirrhosis father cirrh...,1
...,...,...,...,...
8651,8651,Z7869100,story bilateral knee arthroscopy u vasectomy f...,1
8652,8652,Z8631790,female who was evaluated today due to language...,2
8653,8653,Z15536786,list items addressed this visit mild cognitive...,2
8654,8654,Z7156572,ors or hesitation language was fluent without ...,0


In [14]:
X = train_df["sequence"]
y = train_df["label"]

y_label = y.to_numpy()
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X,y,random_state=0,test_size=0.1, stratify=y_label)

y_train_valid_label = y_train_valid.to_numpy()
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, random_state=0, test_size=0.1, stratify=y_train_valid_label)

In [60]:
print(len(X_train))
print(len(X_test))
print(len(X_valid))

7011
866
780


In [16]:
print(X_train)

5988    hasic errors followed simple and complex comma...
4844    minutes away patient is the hcppoa for his fat...
1703    has boston senior health care and laundry serv...
7053    ve family history mother died at father metast...
2057    ii dm cb retinopathy npdr od pdr os sp panreti...
                              ...                        
2353    drinking in social history social history narr...
2091    started celexa in after death of father she wa...
458     devoid of acute suicidal homicidal or psychoti...
5039    standard drinks comment raresocial u drug use ...
7793    will check b and thyroid function recent mri f...
Name: sequence, Length: 7011, dtype: object


In [17]:
y_train.value_counts()

1    3793
0    1972
2    1246
Name: label, dtype: int64

In [124]:
# Initialize a CountVectorizer object: count_vectorizer
count_vectorizer = CountVectorizer(stop_words="english",analyzer='word')

# Transform the training data using only the 'text' column values: count_train 
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)
count_valid = count_vectorizer.transform(X_valid)
#count_tanish = count_vectorizer.transform(tanish_df["sequence"].to_numpy())

# Create the CountVectorizer DataFrame: count_train
count_train = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())

# Create the CountVectorizer DataFrame: test_df
count_test = pd.DataFrame(count_test.A, columns=count_vectorizer.get_feature_names())

count_valid = pd.DataFrame(count_valid.A, columns=count_vectorizer.get_feature_names())

#count_tanish = pd.DataFrame(count_tanish.A, columns=count_vectorizer.get_feature_names())

#count_tanish.head()
count_test.head()

,aaa,aaafpat,aand,aaox,ab,abacavir,abated,abbey,abby,abc,...,zietmen,zing,zocor,zoloft,zolpidem,zoo,zoom,zostavax,zoster,zucker
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:
pickle.dump(count_vectorizer.vocabulary_, open("feature.pkl", "wb"))

In [122]:
len(count_vectorizer.vocabulary_)

10645

In [130]:
tfidf_vectorizer = TfidfVectorizer(stop_words="english",analyzer='word')
tfidf_train= tfidf_vectorizer.fit(X_train)
tfidf_test = tfidf_vectorizer.fit(X_test)
tfidf_valid = tfidf_vectorizer.fit(X_valid)
#tfidf_tanish = tfidf_vectorizer.transform(tanish_df["sequence"].to_numpy())

# Create the TfidfVectorizer DataFrame: tfidf_df
# tfidf_train  = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())
# tfidf_test  = pd.DataFrame(tfidf_test.A, columns=tfidf_vectorizer.get_feature_names())
# tfidf_valid  = pd.DataFrame(tfidf_valid.A, columns=tfidf_vectorizer.get_feature_names())
#tfidf_tanish = pd.DataFrame(tfidf_tanish.A, columns=tfidf_vectorizer.get_feature_names())

#tfidf_valid.isna().sum()

In [132]:
tfidf_valid.get_feature_names()

['14',
 'aaafpat',
 'aaox',
 'abbreviated',
 'abc',
 'abd',
 'abdomen',
 'abdominal',
 'abilities',
 'ability',
 'ablation',
 'able',
 'abnormal',
 'abnormalities',
 'abraczsinskas',
 'abramson',
 'abs',
 'absent',
 'absentminded',
 'abstra',
 'abstract',
 'abstraction',
 'abuse',
 'ac',
 'acarbose',
 'access',
 'accident',
 'accidental',
 'accidents',
 'accompanie',
 'accompanied',
 'according',
 'ace',
 'acei',
 'acetaminophen',
 'achilles',
 'aching',
 'acid',
 'acidosis',
 'acne',
 'act',
 'action',
 'activation',
 'active',
 'activities',
 'activity',
 'acute',
 'ad',
 'adaches',
 'addendum',
 'addition',
 'additional',
 'address',
 'addressed',
 'ade',
 'adenocarcinoma',
 'adenoidectomy',
 'adenoma',
 'adenopathy',
 'adequate',
 'adhd',
 'adhesive',
 'adjustment',
 'adl',
 'adliadl',
 'adls',
 'adlsindependent',
 'admi',
 'administered',
 'admission',
 'admitted',
 'adult',
 'advance',
 'advanced',
 'advancing',
 'advised',
 'aerobic',
 'afe',
 'afebrile',
 'affe',
 'affect',
 'a

In [125]:
pickle.dump(tfidf_vectorizer, open("feature.pkl", "wb"))

In [82]:
tfidf_train_features_df = pd.concat([tfidf_train, y_train.reset_index(drop=True)], axis=1) #ONLY run once

In [83]:
tfidf_test_features_df = pd.concat([tfidf_test, y_test.reset_index(drop = True)], axis = 1)

In [84]:
tfidf_valid_features_df = pd.concat([tfidf_valid, y_valid.reset_index(drop = True)], axis = 1)

In [85]:
tfidf_train_features_df.shape

(7011, 10646)

In [86]:
tfidf_train_features_df_data = tfidf_train_features_df.values
X_smote, y_smote = tfidf_train_features_df_data[:, :-1], tfidf_train_features_df_data[:, -1]

tfidf_test_features_df_data = tfidf_test_features_df.values
X_smote_test, y_smote_test = tfidf_test_features_df_data[:, :-1], tfidf_test_features_df_data[:, -1]

tfidf_valid_features_df_data = tfidf_valid_features_df.values
X_smote_valid, y_smote_valid = tfidf_valid_features_df_data[:, :-1], tfidf_valid_features_df_data[:, -1]

In [87]:
#X_smote_test, y_smote_test = 
oversample = SMOTE()
X_train_new, y_train_new = oversample.fit_resample(X_smote, y_smote)

counter = Counter(y_train_new)
print("Train Data after SMOTE")
for k,v in counter.items():
    print('Class=%d, n=%d' % (k, v))

Train Data after SMOTE
Class=0, n=3793
Class=1, n=3793
Class=2, n=3793


In [88]:
oversample2 = SMOTE()
X_test_new, y_test_new = oversample2.fit_resample(X_smote_test, y_smote_test)
counter2 = Counter(y_test_new)
print("Test Data after SMOTE")
for k,v in counter2.items():
    print('Class=%d, n=%d' % (k, v))

Test Data after SMOTE
Class=1, n=469
Class=2, n=469
Class=0, n=469


In [89]:
oversample3 = SMOTE()
X_valid_new, y_valid_new = oversample3.fit_resample(X_smote_valid, y_smote_valid)
counter3 = Counter(y_valid_new)
print("Test Data after SMOTE")
for k,v in counter3.items():
    print('Class=%d, n=%d' % (k, v))

Test Data after SMOTE
Class=1, n=421
Class=2, n=421
Class=0, n=421


In [90]:
X_train_new = pd.DataFrame(X_train_new)
y_train_new = pd.Series(y_train_new)

X_test_new = pd.DataFrame(X_test_new)
y_test_new = pd.Series(y_test_new)

X_valid_new = pd.DataFrame(X_valid_new)
y_valid_new = pd.Series(y_valid_new)

print(type(y_valid_new))

<class 'pandas.core.series.Series'>


In [91]:
tfidf_train_features_df_new = pd.concat([X_train_new, y_train_new.reset_index(drop=True)], axis=1)
tfidf_train_features_df_new.columns = tfidf_train_features_df.columns

tfidf_test_features_df_new = pd.concat([X_test_new, y_test_new.reset_index(drop=True)], axis=1)
tfidf_test_features_df_new.columns = tfidf_test_features_df.columns

tfidf_valid_features_df_new = pd.concat([X_valid_new, y_valid_new.reset_index(drop=True)], axis=1)
tfidf_valid_features_df_new.columns = tfidf_valid_features_df.columns

In [24]:
tfidf_train_features_df_new = tfidf_train_features_df
tfidf_test_features_df_new = tfidf_test_features_df
tfidf_valid_features_df_new = tfidf_valid_features_df

In [92]:
mappings = {0.0 : 0, 1.0 : 1, 2.0 : 2}

tfidf_train_features_df_new.label = [mappings[item] for item in tfidf_train_features_df_new.label]
tfidf_test_features_df_new.label = [mappings[item] for item in tfidf_test_features_df_new.label]
tfidf_valid_features_df_new.label = [mappings[item] for item in tfidf_valid_features_df_new.label]

# tfidf_train_features_df_new.to_csv("Data/SMOTE_train.csv")
# tfidf_test_features_df_new.to_csv("Data/SMOTE_test.csv")
# tfidf_valid_features_df_new.to_csv("Data/SMOTE_val.csv")

tfidf_features_master = pd.concat([tfidf_train_features_df_new,tfidf_test_features_df_new,tfidf_valid_features_df_new])

tfidf_features_master

,aaa,aaafpat,aand,aaox,ab,abacavir,abated,abbey,abby,abc,...,zing,zocor,zoloft,zolpidem,zoo,zoom,zostavax,zoster,zucker,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [93]:
tfidf_train_features_df_new.drop(columns = 'label')
tfidf_test_features_df_new.drop(columns = 'label')
tfidf_valid_features_df_new.drop(columns = 'label')

,aaa,aaafpat,aand,aaox,ab,abacavir,abated,abbey,abby,abc,...,zietmen,zing,zocor,zoloft,zolpidem,zoo,zoom,zostavax,zoster,zucker
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
def filter_features_by_cor(df):
    m = len(df.columns)
    output = df.iloc[:,m-1] 
    output_list = output.tolist()
    corrcoef_array = []

    for i in range(0,m-2):
        input_list = df.iloc[:,i].tolist()
        cols = [input_list, output_list]
        corrcoef = abs(np.corrcoef(cols)) 
        #print(type(corrcoef))
        corrcoef_array = np.append(corrcoef_array,corrcoef[0,1])

    feature_names = list(df)
    feature_names = feature_names[0:m-2]
    output_df = pd.DataFrame(feature_names, columns=['Features'])
    output_df['CorrCoef'] = corrcoef_array
    output_df = output_df.sort_values('CorrCoef')
    output_df = output_df.reset_index()
    output_df = output_df.drop(columns = "index")
    
    return output_df

In [96]:
tfidf_output_df = filter_features_by_cor(tfidf_train_features_df_new)
tfidf_output_df = tfidf_output_df.sort_values(by=['CorrCoef'],ascending = False)
#tfidf_output_df.to_csv('Data/tfidf_vector_feature_corr.csv')

tfidf_output_df

,Features,CorrCoef
10643,intact,0.630383
10642,oriented,0.510477
10641,orientationsensorium,0.402591
10640,reports,0.382848
10639,immediate,0.371857
...,...,...
15,wassociated,0.000000
16,artment,0.000000
17,heaviness,0.000000
18,ventolin,0.000000


In [116]:
# Setting Correlation threshold
top_tfidf_features_df = tfidf_output_df#[:500]
filtered_tfidf_train = tfidf_train_features_df_new.filter(items=top_tfidf_features_df['Features'])
filtered_tfidf_test = tfidf_test_features_df_new.filter(items=top_tfidf_features_df['Features'])
filtered_tfidf_valid   = tfidf_valid_features_df_new.filter(items=top_tfidf_features_df['Features'])
#filtered_tfidf_tanish = tfidf_tanish.filter(items = top_tfidf_features_df['Features'])

In [ ]:
filtered_tfidf_train.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Data\tfidf_train_no_smote.csv", index = False)
filtered_tfidf_test.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Data\tfidf_test_no_smote.csv", index = False)
filtered_tfidf_valid.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Data\tfidf_validation_no_smote.csv", index = False)

In [31]:
y_train.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Data\y_train_no_smote.csv", index = False)
y_test.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Data\y_test_no_smote.csv", index = False)
y_valid.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Data\y_validation_no_smote.csv", index = False)

In [ ]:
c = 0.1

svm = SVC(probability = True, C=c, kernel='linear',  random_state=42)
svm.fit(filtered_tfidf_train, y_train_new)

print("C:", c, "\n")

scores = cross_val_score(svm, tfidf_features_master, tfidf_features_master["label"], cv=5)

y_prob_svm = svm.predict_proba(filtered_tfidf_valid)
labels_svm = svm.predict(filtered_tfidf_valid)

target_names = ['NO', 'NTR', 'YES']
results_svm = classification_report(y_valid_new, labels_svm, target_names=target_names, output_dict=True)
results_svm = pd.DataFrame(results_svm).transpose()

#     results_svm_tanish = classification_report(real_tanish_labels, tanish_svm_labels, target_names=target_names, output_dict=True)
#     results_svm_tanish = pd.DataFrame(results_svm_tanish).transpose()

print(results_svm)
print("CV Scores: ", scores)
print("AUC: ", roc_auc_score(y_valid_new, y_prob_svm, multi_class = 'ovr'))
print("ACC: ", accuracy_score(y_valid_new, labels_svm))
print("\nConfusion Matrix: \n", confusion_matrix(y_valid_new, labels_svm))
print("---------------------------------------------------------------------\n")
    
# y_prob_svm_tanish = svm.predict_proba(filtered_tfidf_tanish)
# tanish_svm_labels = svm.predict(filtered_tfidf_tanish)
# real_tanish_labels = tanish_df["label"].to_numpy()

# print("\nTanish Data: \n")
# print(results_svm_tanish)
# print("AUC: ", roc_auc_score(real_tanish_labels, y_prob_svm_tanish, multi_class='ovr'))
# print("\nConfusion Matrix: \n", confusion_matrix(real_tanish_labels, tanish_svm_labels))

C: 0.1 



In [117]:
c = 0.1
labels_lgr_tfidf = np.array([])

lr = LogisticRegression(C = c, random_state = 42)
lr.fit(filtered_tfidf_train, y_train_new)

#scores = cross_val_score(lr, tfidf_features_master, tfidf_features_master["label"], cv=5)
#print("CV:", scores)

LogisticRegression(C=0.1, random_state=42)

In [118]:
lr.score(filtered_tfidf_valid, y_valid_new)

0.9588281868566905

In [119]:
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("feature.pkl", "rb")))
test = tfidf_vectorizer.fit_transform(np.array([" ------- School of Drama. His daugher is a pediatric nurse practitioner at Longwood Pediatrics. EtOH: None x 14 years. Occupation: Now retired-but looking to go back to work in human resources. Exercise: Plans to restart Family History Alcoholism Relative: Father Relative: Paternal Half-Sister Alzheimer's Disease Relative: Mother [Negative] Colon Cancer [Negative] Coronary Artery Disease Diabetes Mell ------- rinary: Testes normal and penis normal. Right testis shows no mass. Left testis shows no mass. Circumcised. Musculoskeletal: Normal range of motion. He exhibits no edema or tenderness. Lymphadenopathy: He has no cervical adenopathy. Neurological: He is alert and oriented to person, place, and time. No cranial nerve deficit. Skin: Skin is warm and dry. No rash noted. No erythema. Psychia"]))

In [120]:
test

<1x73 sparse matrix of type '<class 'numpy.float64'>'
	with 73 stored elements in Compressed Sparse Row format>

In [121]:
y_prob_lr = lr.predict_proba(filtered_tfidf_valid)
#print("C:", c, "\n: ", )

labels_lgr_tfidf = lr.predict(test)

#     y_prob_lgr_tanish = lr.predict_proba(filtered_tfidf_tanish)

#     tanish_lr_labels = lr.predict(filtered_tfidf_tanish)
#     real_tanish_labels = tanish_df["label"].to_numpy()

#     results_lr_tanish = classification_report(real_tanish_labels, tanish_lr_labels, target_names=target_names, output_dict=True)
#     results_lr_tanish = pd.DataFrame(results_lr_tanish).transpose()

ValueError: X has 73 features per sample; expecting 10644

In [54]:
target_names = ['NO', 'NTR', 'YES']

results_lgr = classification_report(y_valid_new, labels_lgr_tfidf, target_names=target_names, output_dict=True)
results_lgr = pd.DataFrame(results_lgr).transpose()

print("\nC: ", c, "\n", results_lgr)
print("AUC: ", roc_auc_score(y_valid_new, y_prob_lr, multi_class='ovr'))
print("ACC: ", accuracy_score(y_valid_new, labels_lgr_tfidf))
#print("MCC: ", matthews_corrcoef(y_valid, y_prob_lr))
print("\nConfusion Matrix: \n", confusion_matrix(y_valid_new, labels_lgr_tfidf))
#     print("\nTanish Data: \n", )
#     print(results_lr_tanish)
#     print("AUC: ", roc_auc_score(real_tanish_labels, y_prob_lgr_tanish, multi_class='ovr'))
#     print("\nConfusion Matrix: \n", confusion_matrix(real_tanish_labels, tanish_lr_labels))
print("---------------------------------------------------------------------\n")


C:  0.1 
               precision    recall  f1-score      support
NO             0.954436  0.945368  0.949881   421.000000
NTR            0.903297  0.976247  0.938356   421.000000
YES            0.953964  0.885986  0.918719   421.000000
accuracy       0.935867  0.935867  0.935867     0.935867
macro avg      0.937232  0.935867  0.935652  1263.000000
weighted avg   0.937232  0.935867  0.935652  1263.000000
AUC:  0.9903492984128954
ACC:  0.9358669833729216

Confusion Matrix: 
 [[398  15   8]
 [  0 411  10]
 [ 19  29 373]]
---------------------------------------------------------------------



In [ ]:
clf_random_forest = RandomForestClassifier()
clf_random_forest.fit(filtered_tfidf_train, y_train_new)

y_prob_rf = clf_random_forest.predict_proba(filtered_tfidf_valid)

labels_rf_tfidf = clf_random_forest.predict(filtered_tfidf_valid)

results_rf = classification_report(y_valid_new, labels_rf_tfidf, target_names=target_names, output_dict=True)
results_rf = pd.DataFrame(results_rf).transpose()

print(results_rf)

scores = cross_val_score(clf_random_forest, tfidf_features_master, tfidf_features_master["label"], cv=5)
print("CV:", scores)

print("AUC: ", roc_auc_score(y_valid_new, y_prob_rf, multi_class='ovr'))
print("ACC: ", accuracy_score(y_valid_new, labels_rf_tfidf))
#print("MCC: ", matthews_corrcoef(y_valid, y_prob_lr))
print("\nConfusion Matrix: \n", confusion_matrix(y_valid_new, labels_rf_tfidf))

# print("\nTanish Data: \n")

# y_prob_rf_tanish = clf_random_forest.predict_proba(filtered_tfidf_tanish)

# tanish_rf_labels = clf_random_forest.predict(filtered_tfidf_tanish)
# real_tanish_labels = tanish_df["label"].to_numpy()

# results_rf_tanish = classification_report(real_tanish_labels, tanish_rf_labels, target_names=target_names, output_dict=True)
# results_rf_tanish = pd.DataFrame(results_rf_tanish).transpose()
# print(results_rf_tanish)

# print("AUC: ", roc_auc_score(real_tanish_labels, y_prob_rf_tanish, multi_class='ovr'))
# print()
# #print("MCC: ", matthews_corrcoef(y_valid, y_prob_lr))
# print("\nConfusion Matrix: \n", confusion_matrix(real_tanish_labels, tanish_rf_labels))